## Imports

In [17]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import cv2

## Selected Data & their (sub)directories
15 species

In [18]:
# The list of chosen birds

birds = 'FLAMINGO, OKINAWA RAIL, NORTHERN PARULA, OVENBIRD, SUPERB STARLING, WALL CREAPER, RED NAPED TROGON, NORTHERN JACANA, MAGPIE GOOSE, IVORY GULL, KOOKABURRA, KILLDEAR, PARADISE TANAGER, RED BELLIED PITTA, RUDY KINGFISHER, STRAWBERRY FINCH'
birds = birds.split(', ')
birds = sorted(birds)

# birds = ['IVORY GULL', 'KILLDEAR', 'KOOKABURRA', ...]
# len(birds) = 15
len(birds)

16

In [3]:
# List of directories of all of the birds

DIR = 'train'
subnames = [name for name in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, name))]

# subanames = [ABBOTTS BABBLER, ABBOTS BOOBY, ...]
# len(subnames) = 400

In [4]:
# List of our chosen species' subdirectories

trainDIR = 'train/'
trainsubs = [trainDIR+str(bird)+'/' for bird in birds]

testDIR = 'test/'
testsubs = [testDIR+str(bird)+'/' for bird in birds]

validDIR = 'valid/'
validsubs = [validDIR+str(bird)+'/' for bird in birds]


# trainsubs = ['train/OKINAWA RAIL/',
#                       ...
#            'train/STRAWBERRY FINCH/']

# testsubs = ['test/OKINAWA RAIL/',
#                       ...
#            'test/STRAWBERRY FINCH/']

# validsubs = ['valid/OKINAWA RAIL/',
#                       ...
#            'valid/STRAWBERRY FINCH/']


In [5]:
# Creating a list of indices for our 15 selected birds, sorted.

indices = []
for i in range(len(subnames)):
    if subnames[i] in birds:
        indices.append(i)
        
# indices = [236, 245, 248, ...]
# len(indices) = 15

## Train/Test Data

In [6]:
# pixel size
pixels = 100

In [7]:
# Creating the list of arrays for our train data. They get normalized.

trainimglist = []
trainindlist = []
for i in range(len(trainsubs)):
    jpgs = next(os.walk(trainsubs[i]))[2][:120] #list of files in the subdir
    for jpg in jpgs:
        img_array = cv2.imread(os.path.join(trainsubs[i],jpg), cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (pixels, pixels))/255.
        trainimglist.append(new_array)
        trainindlist.append(i)
        
        img_array = cv2.imread(os.path.join(trainsubs[i],jpg))
        
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        new_gray = cv2.resize(img_array, (pixels, pixels))/255.
        trainimglist.append(new_gray)
        trainindlist.append(i)
        
        flip = cv2.flip(img_array, 0)
        new_flip = cv2.resize(flip, (pixels, pixels))/255.
        trainimglist.append(new_flip)
        trainindlist.append(i)
        
# len(trainimglist) = 5400            # The list of img arrays
# len(trainindlist) = 5400            # The list of indices associated with each array of same index in trainimglist
len(trainimglist), len(trainindlist)

(5760, 5760)

In [8]:
# Creating the list of arrays for our test data. They get normalized, too.

testimglist = []
testindlist = []
for i in range(len(testsubs)):
    jpgs = next(os.walk(testsubs[i]))[2] #list of files in the subdir
    for jpg in jpgs:
        img_array = cv2.imread(os.path.join(testsubs[i],jpg), cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (pixels, pixels))/255.
        testimglist.append(new_array)
        testindlist.append(i)
# len(testimglist) = 75            # The list of img arrays
# len(testindlist) = 75            # The list of indices associated with each array of same index in imglist

In [9]:
# Creating the list of arrays for our validation data. They get normalized, too.

validimglist = []
validindlist = []
for i in range(len(validsubs)):
    jpgs = next(os.walk(validsubs[i]))[2] #list of files in the subdir
    for jpg in jpgs:
        img_array = cv2.imread(os.path.join(validsubs[i],jpg), cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (pixels, pixels))/255.
        validimglist.append(new_array)
        validindlist.append(i)
        

In [10]:
# Changing our train/test data to numpy arrays

X_train = np.array(trainimglist)
y_train = np.array(trainindlist)
X_test = np.array(testimglist)
y_test = np.array(testindlist)
X_valid = np.array(testimglist)
y_valid = np.array(testindlist)

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_valid.shape, y_valid.shape


((5760, 100, 100, 3),
 (5760,),
 (80, 100, 100, 3),
 (80,),
 (80, 100, 100, 3),
 (80,))

## Current best model

In [15]:
# Current best model -- Use the next cells below to make changes and print new results while keeping these ones. Update 
# Delete these when we get a better one if desired.

from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

X_train = np.array(trainimglist)
y_train = np.array(trainindlist)
X_test = np.array(testimglist)
y_test = np.array(testindlist)
X_valid = np.array(testimglist)
y_valid = np.array(testindlist)


num_classes = 16
input_shape = (pixels,pixels,3)

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
X_valid = np.expand_dims(X_valid, -1)
print("X_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# 4 x [ Conv2D + MaxPooling2D ] layers
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)), 
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

X_train shape: (5760, 100, 100, 3, 1)
5760 train samples
80 test samples
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 21, 21, 128)       73856     
                                               

In [16]:
batch_size = 100
epochs = 10
val_split = 0.1
early_stopping = [
    EarlyStopping(monitor='val_loss', patience=4), 
    EarlyStopping(monitor='accuracy', patience=4, min_delta=.02)
]

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=val_split, callbacks=None)
#Change val_split for more traindata




Epoch 1/10
52/52 [==============================] - 26s 490ms/step - loss: 2.1669 - accuracy: 0.2799 - val_loss: 9.5306 - val_accuracy: 0.0243
Epoch 2/10
52/52 [==============================] - 26s 494ms/step - loss: 1.3761 - accuracy: 0.5527 - val_loss: 8.6452 - val_accuracy: 0.1094
Epoch 3/10
52/52 [==============================] - 27s 516ms/step - loss: 0.9464 - accuracy: 0.7051 - val_loss: 8.1146 - val_accuracy: 0.2830
Epoch 4/10
52/52 [==============================] - 27s 523ms/step - loss: 0.6722 - accuracy: 0.7803 - val_loss: 9.5775 - val_accuracy: 0.1302
Epoch 5/10
52/52 [==============================] - 29s 550ms/step - loss: 0.5356 - accuracy: 0.8308 - val_loss: 8.5895 - val_accuracy: 0.3142
Epoch 6/10
52/52 [==============================] - 29s 555ms/step - loss: 0.3983 - accuracy: 0.8715 - val_loss: 10.5301 - val_accuracy: 0.2726
Epoch 7/10
52/52 [==============================] - 28s 543ms/step - loss: 0.3149 - accuracy: 0.8951 - val_loss: 10.0520 - val_accuracy: 0.31

In [ ]:
model.save('flamingo')

In [ ]:
model23 = tf.keras.models.load_model('flamingo')
score_v = model23.evaluate(X_valid, y_valid)
print(score_v[0])
print(score_v[1])
score_v2 = model23.evaluate(X_test, y_test)
print(score_v2[0])
print(score_v2[1])

## Model with a chip on its shoulder

In [ ]:
# # Have to reinitialize our train/test vars because running the model modifies it
# X_train = np.array(trainimglist)
# y_train = np.array(trainindlist)
# X_test = np.array(testimglist)
# y_test = np.array(testindlist)


# num_classes = 15
# input_shape = (80,80,3)

# X_train = np.expand_dims(X_train, -1)
# X_test = np.expand_dims(X_test, -1)
# print("X_train shape:", X_train.shape)
# print(X_train.shape[0], "train samples")
# print(X_test.shape[0], "test samples")

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# # Use different layers here!!!
# model = keras.Sequential(
#     [
#         keras.Input(shape=input_shape),
#         layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)), 
#         layers.Flatten(),
#         layers.Dropout(0.5),
#         layers.Dense(num_classes, activation="softmax"),
#     ] layers.
# )
# model.summary()